In [1]:
import itertools
import cv2
import otsu
import matplotlib.pyplot as plt
import numpy as np
import glob
import math
from PlotToImage import PlotToImage
from auto_tsmo import full_method
import time
import json

In [2]:
def ImShow(im, size=(14,17)):
    plt.figure(figsize = size)
    plt.imshow(im, 'gray')
    plt.show()

def HistPlot(vals, size=(14,17)):
    plt.figure(figsize = size)
    plt.plot(np.arange(len(vals)), vals)
    plt.show()

In [3]:
def symm(array, perc = .11):
    array /= np.max(array)
    
    pairs = []
    for i in range(len(array)):
        for j in range(i, min(i + len(array)//3, len(array))):
            if abs(array[i]-array[j]) < perc and (array[i] != 0 and array[j] != 0):
                pairs.append([i,j])
    sym = pairs
    
    new_dens = np.ones(len(array))

    for i in range(len(sym)):
        if (sym[i][0] + sym[i][1]) % 2 == 0:
            k = int(((sym[i][1] - sym[i][0])/2) + sym[i][0])
    #         print(k)
            new_dens[k] *= 1 + (array[k]/max(array))
        else:
            k = int((((sym[i][1] - sym[i][0])/2) - .5) + sym[i][0])
    #         print(k)
            new_dens[k] *= 1+ (array[k]/max(array))
            new_dens[k+1] *= 1 + (array[k]/max(array))
    
    return new_dens > 1

In [4]:
def whiteDensity(im, n = 1, val = 254):
    new_im = cv2.threshold(im, val, 255, cv2.THRESH_BINARY)[1]
    den = [np.sum([np.sum([new_im[j,i] for j in range(im.shape[0])]) for i in range(k, k+n)]) for k in range(0, im.shape[1], n)]
    den /= np.max(den)
    return den

In [5]:
def grayDensity(im, n = 1):
    new_im = im
    den = [np.sum([np.sum([new_im[j,i] for j in range(im.shape[0])]) for i in range(k, k+n)]) for k in range(0, im.shape[1], n)]
    den /= np.max(den)
    return den

In [6]:
def isCorrect(reg):
    den = grayDensity(reg)
    min_val = den.min()
    int_val = reg.sum() / (reg.shape[0] * reg.shape[1])
  #  print(int_val)
    if min_val < 0.75:
        if int_val > 80:
            return True
        
    return False

In [7]:
from scipy.ndimage.filters import gaussian_filter1d as gauss
def getImMask(im):
    return gauss(whiteDensity(im, val = thresh_val), 5) > 0.01

In [8]:
def getRegionsAndMasks(im, im_mask, return_regions = True):
    reg_masks = []
    mask = []
    for i, s in enumerate(im_mask):
        if s:
            mask.extend([i])
        else:
            if len(mask) > 5:
                reg_masks.append(mask)
            mask = []
    if len(mask) > 5:
        reg_masks.append(mask)
    if not return_regions:
        return reg_masks
    regions = [np.array([[im[i,j] for j in mask] for i in range(im.shape[0])], dtype=np.uint8) for mask in reg_masks]
    return regions, reg_masks

def drawContours(frame, regions, reg_masks):
    global current_frame
    global predictions
    for i, reg in enumerate(regions):
      #  ImShow(reg)

        thresholds, masks = full_method(reg, L = 2**8, M=128)
        #for j, mask in enumerate(masks):
        #    print(j)
         #   ImShow(mask)
       # ImShow(masks[-2])

        thresh = (((masks[-2]+masks[-1])*255)).astype(np.uint8)
        #ImShow(thresh)
     #   thresh = np.array((reg > thresh_val)*255).astype(np.uint8)
    #    cv2.imshow("thresh"+str(i), thresh)
        kernel = np.ones((3,3), np.uint8) 
        thresh = cv2.dilate(thresh, kernel, iterations=10) 
      #  ImShow(thresh)

        contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for cnt in contours:
            x,y,w,h = cv2.boundingRect(cnt)
         #   if not isCorrect(reg[y:y+h,x:x+w]):
          #      continue
            x += reg_masks[i][0]
            if w > 10 and h > 20 and h / w > 0.5 and h / w < 3:
               # if isCorrect(reg[y:y+h,x-sym_mask[i][0]:x-sym_mask[i][0]+w]):
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)
#                     if current_frame in predictions:
#                         predictions[current_frame].append([x, y, w, h])
#                     else:
#                         predictions[current_frame] = []
#                         predictions[current_frame].append([x, y, w, h])
      #          else:
       #             cv2.rectangle(frame, (x, y), (x+w, y+h), (0,0,255), 2)
      #  plt.imshow(frame)
       # plt.show()

In [9]:
# start = 12
# curr = 0
# for d in data:
#     curr += 1
#     if (curr < start):
#         continue
#     frame = d.copy()

#     im = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     cv2.imshow("im", im)
    
#     den = whiteDensity(im)
#     im_mask = gauss(den, 5) > 0.01
#  #   cv2.imshow("mask_im", mask_im)
#     hor_regs, hor_masks = getRegions(im, im_mask)
    
#     for i, hreg in enumerate(hor_regs):
#         im = np.transpose(hreg)
#         den = whiteDensity(im)
#         im_mask = gauss(den, 5) > 0
#         vert_regs, vert_masks = getRegions(im, im_mask)
        
#         for j, vreg in enumerate(vert_regs):
#             vreg = np.transpose(vreg)
#             h, w = vreg.shape[0], vreg.shape[1]
#             x, y = hor_masks[i][0], vert_masks[j][0]
            
#             cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)
        
# #     for reg in regions:
# #         reg = np.transpose(reg)
# #         ImShow(reg)
# #         den = whiteDensity(reg)
# #         den = gauss(den, 5)
# #        # HistPlot(den)
# #         mask = den > 0.01
# #         _regions, _mask = getRegions(reg, mask)
# #         for _reg in _regions:
# #             ImShow(np.transpose(_reg))
#  #   drawContours(frame, regions, sym_mask)
    
    
#     cv2.imshow('frame', frame)
  
    
#     if cv2.waitKey(100000) & 0xFF == ord('q'):
#         break
# cv2.destroyAllWindows()

In [10]:
# import time
# video_path = r'C:\Users\admin\Google\HumanDetection\Data\video.mp4'
# cap = cv2.VideoCapture(video_path)

# start = time.time()

# first_frame = 1100
# curr = 0

# while(cap.isOpened()):
#     curr += 1
    
#     ret, frame = cap.read()
#    # frame = cv2.resize(frame, (frame.shape[1]//2,frame.shape[0]//2))
    
#     if (curr < first_frame):
#         start = time.time()
#         continue
    
#     frame = cv2.resize(frame, (512, 512))
#     im = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     cv2.imshow("frame", im)
     
#     step = 512 // 8
#     SIZE = (6,7)
#     for i in range(0, 512, step):
#         for j in range(0, 512, step):
#             reg = im[j:j+step,i:i+step]
#             den = density(reg)
#             if den.min() < 0.75:
#                 if reg.sum() > 70 * step**2:
#                     overlay = frame.copy()
#                     cv2.rectangle(overlay, (i, j), (i+step, j+step), (0, 255, 0), -1) 
#                     alpha = 0.4 
#                     frame = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)
#                 else:
#                     a = 1
#                     overlay = frame.copy()
#                     cv2.rectangle(overlay, (i, j), (i+step, j+step), (0, 0, 255), -1) 
#                     alpha = 0.4 
#                     frame = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)
                    
#     cv2.imshow('contours', frame)
    
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
        
# end = time.time()

# print("Average FPS:", (curr-first_frame)/(end-start))

# cap.release()
# cv2.destroyAllWindows()

In [11]:
video_dataset = {}
with open("video_groundtruth.json", "r") as read_file:
    video_dataset = json.load(read_file)
FLIR_dataset = {}
with open("FLIR_val_groundtruth.json", "r") as read_file:
    FLIR_dataset = json.load(read_file)
# FLIR_dataset = {}
# with open("FLIR_train_groundtruth.json", "r") as read_file:
#     FLIR_dataset = json.load(read_file)

In [12]:
all_data = np.append(list(video_dataset.keys()), list(FLIR_dataset.keys()))
np.random.shuffle(all_data)
all_data = all_data[:100]
# all_data

In [13]:
data = [cv2.imread(ad) for ad in all_data]

In [14]:
video_predictions = {}
FLIR_val_predictions = {}

In [15]:
def getRects(regions, reg_masks):
    filtered_rects = []
    for i, reg in enumerate(regions):
        
        
        ImShow(reg)


        
        thresholds, masks = full_method(reg, L = 2**8, M=128)
        
        for j, mask in enumerate(masks):
            print(j)
            ImShow(mask)
        ImShow(masks[-2])
        
        thresh = (((masks[-2]+masks[-1])*255)).astype(np.uint8)
        kernel = np.ones((3,3), np.uint8) 
        thresh = cv2.dilate(thresh, kernel, iterations=10) 
        ImShow(thresh)
        
        contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for cnt in contours:
            x,y,w,h = cv2.boundingRect(cnt)
         #   if not isCorrect(reg[y:y+h,x:x+w]):
          #      continue
            x += reg_masks[i][0]
            if w > 10 and h > 20 and h / w > 0.5 and h / w < 3:
                filtered_rects.append([x,y,w,h])
    return filtered_rects 

def drawRects(frame, rects):
    for rect in rects:
        x,y,w,h = rect
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)

def saveRects(rects, im_path, predictions):
    for rect in rects:
        if im_path in predictions:
            predictions[im_path].append(rect)
        else:
            predictions[im_path] = []
            predictions[im_path].append(rect)

In [16]:
def getRects(regions, reg_masks):
    filtered_cnts = []
    for i, reg in enumerate(regions):
        thresholds, masks = full_method(reg, L = 2**8, M=128)
        thresh = (((masks[-2]+masks[-1])*255)).astype(np.uint8)
        kernel = np.ones((3,3), np.uint8) 
        thresh = cv2.dilate(thresh, kernel, iterations=10) 
        
        contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for cnt in contours:
            x,y,w,h = cv2.boundingRect(cnt)
         #   if not isCorrect(reg[y:y+h,x:x+w]):
          #      continue
            x += reg_masks[i][0]
            if w > 10 and h > 20 and h / w > 0.5 and h / w < 3:
                filtered_cnts.append([x,y,w,h])
    return filtered_cnts 


def drawContours(frame, rects):
    for rect in rects:
        x,y,w,h = rect
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)

In [15]:
for im_path in list(video_dataset.keys()):
    frame = cv2.imread(im_path)
    im = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    cv2.imshow("im", im)
    
    
    thresholds, masks = full_method(im, L = 2**8, M=128)
    N = 8
    thresh_val = thresholds[-1]
    den = whiteDensity(im, n = N, val = thresh_val)  

    sym = symm(den, 0.1)
    sym = np.array([[sym[i] for j in range(N)] for i in range(len(sym))]).reshape(-1,) 
#     sym = getImMask(im)
    sym_im = np.array([[im[i,j] for j in range(im.shape[1])]*sym for i in range(im.shape[0])], dtype = np.uint8)
    cv2.imshow("symm_im", sym_im)

    
    
    regions, reg_masks = getRegionsAndMasks(im, sym)
    rects = getRects(regions, reg_masks)
    drawRects(frame, rects)
    saveRects(rects, im_path, video_predictions)
    
#     drawContours(frame, regions, reg_masks)    
    
    cv2.imshow('frame', frame)
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()

In [38]:
with open('video_predictions.json', 'w') as outfile:
    json.dump(video_predictions, outfile)

In [17]:
ind = 0
for im_path in list(FLIR_dataset.keys()):
    ind += 1
    if (ind < 10):
        continue
    frame = cv2.imread(im_path)
    im = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    cv2.imshow("im", im)
    
    
    thresholds, masks = full_method(im, L = 2**8, M=128)
    N = 8
    thresh_val = thresholds[-1]
    den = whiteDensity(im, n = N, val = thresh_val)  

    sym = symm(den, 0.1)
    sym = np.array([[sym[i] for j in range(N)] for i in range(len(sym))]).reshape(-1,) 
#     sym = getImMask(im)
    sym_im = np.array([[im[i,j] for j in range(im.shape[1])]*sym for i in range(im.shape[0])], dtype = np.uint8)
    cv2.imshow("symm_im", sym_im)

    
    
    regions, reg_masks = getRegionsAndMasks(im, sym)
    rects = getRects(regions, reg_masks)
    drawRects(frame, rects)
    saveRects(rects, im_path, FLIR_val_predictions)
    
#     drawContours(frame, regions, reg_masks)    
    
    cv2.imshow('frame', frame)
    
    
    if cv2.waitKey(10000) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()

In [31]:
with open('FLIR_val_predictions.json', 'w') as outfile:
    json.dump(FLIR_val_predictions, outfile)

In [18]:
start = 0
curr = 0
for d in data:
    curr += 1
    if (curr < start):
        continue
    frame = d.copy()
   # cv2.imshow('zero', frame)
    im = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow("im", im)
    
    

    
    
    if cv2.waitKey(100000) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()

In [16]:
start = 0
curr = 0
for d in data:
    curr += 1
    if (curr < start):
        continue
    frame = d.copy()
   # cv2.imshow('zero', frame)
    im = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow("im", im)
    
    
    thresholds, masks = full_method(im, L = 2**8, M=128)
    N = 8
    thresh_val = thresholds[-1]
    den = whiteDensity(im, n = N, val = thresh_val)  

    sym = symm(den, 0.1)
    sym = np.array([[sym[i] for j in range(N)] for i in range(len(sym))]).reshape(-1,) 
    sym = getImMask(im)
    sym_im = np.array([[im[i,j] for j in range(im.shape[1])]*sym for i in range(im.shape[0])], dtype = np.uint8)
    cv2.imshow("symm_im", sym_im)

    
    
    regions, reg_masks = getRegionsAndMasks(im, sym)
    drawContours(frame, regions, reg_masks)
    
    
    cv2.imshow('frame', frame)
    
    
    if cv2.waitKey(10000) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()

NameError: name 'data' is not defined

In [11]:
video_path = r'C:\Users\admin\Google\HumanDetection\Data\video.mp4'
cap = cv2.VideoCapture(video_path)

start = time.time()

predictions = {}
first_frame = 1250
current_frame = -1

while(cap.isOpened()):
    current_frame += 1
    
    ret, frame = cap.read()
    
    if (current_frame < first_frame):
        start = time.time()
        continue
    
    if (current_frame == 1500):
        break
    
#    frame = cv2.resize(frame, (frame.shape[1]//2,frame.shape[0]//2))

    im = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow("im", im)
    
    
    thresholds, masks = full_method(im, L = 2**8, M=128)
    N = 8
    thresh_val = thresholds[-1]
    den = whiteDensity(im, n = N, val = thresh_val)  

    sym = symm(den, 0.1)
    sym = np.array([[sym[i] for j in range(N)] for i in range(len(sym))]).reshape(-1,)
    
    sym_im = np.array([[im[i,j] for j in range(im.shape[1])]*sym for i in range(im.shape[0])], dtype = np.uint8)
    cv2.imshow("symm_im", sym_im)

    regions, reg_masks = getRegionsAndMasks(im, sym)
    drawContours(frame, regions, reg_masks)
    
    with open('predictions.json', 'w') as outfile:
        json.dump(predictions, outfile)
    
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
end = time.time()

print("Average FPS:", (current_frame-first_frame)/(end-start))


cap.release()
cv2.destroyAllWindows()

Average FPS: 0.9592697599936474
